In [ ]:
# Install MDAnalysis
!pip install --upgrade MDAnalysis
!pip install py3Dmol

In [ ]:
import requests
import MDAnalysis as mda
import matplotlib.pyplot as plt
import numpy as np
from MDAnalysis.analysis import rms
from MDAnalysis.analysis import align, rms
import py3Dmol
from tqdm import tqdm

In [ ]:
# Download the MD trajectory
url = 'https://github.com/durrantlab/colab-support-files/raw/main/hxk2_sim.zip'
file_path = 'hxk2_sim.zip'
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the file in binary write mode and save the content to the file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully and saved as {file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
# Uncompress the file
!rm -f hxk2_first_frame.pdb hxk2.dcd
!unzip hxk2_sim.zip

In [ ]:
# Load the trajectory

# Create a Universe using the PDB file as the topology and the DCD file as the
# trajectory. The Universe object in MDAnalysis, which represents the entire
# system of the simulation, including atoms, residues, and segments.
u = mda.Universe("hxk2_first_frame.pdb", "hxk2.dcd")

# Now you can perform various analyses on the Universe. For example, print the
# number of atoms in the Universe.
print(f"Number of atoms in the Universe: {len(u.atoms)}")

In [ ]:
# @title Save every tenth frame to single PDB files for quick visualization.

# Loop through each frame in the trajectory
for ts in tqdm(u.trajectory):
    if ts.frame % 10 != 0:
        continue
    # Define the filename for the current frame
    # This creates filenames like "frame_0.pdb", "frame_1.pdb", etc.
    filename = f"frame_{ts.frame}.pdb"

    # Initialize a writer for each frame with the correct number of atoms
    with mda.Writer(filename, u.atoms.n_atoms) as W:
        # Write the current frame to a PDB file
        W.write(u.atoms)


In [ ]:
# @title Jacob Durrant's super simple trajectory visualizer
frame = 410 # @param {type:"slider", min:0, max:430, step:10}
# Visualize the trajectory

view = py3Dmol.view(width=800, height=400)
view.addModel(open(f'frame_{frame}.pdb', 'r').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'white', 'opacity':0.75}})
view.addStyle({'resn': 'LIG'}, {'stick': {'radius':0.75}})
# view.zoomTo({'resn': 'LIG'})
view.zoomTo()
view.show()


In [ ]:
# Calculate the RMSD of the backbone atoms between the first frame of the
# trajectory and each subsequent frame.

# Select the protein backbone
backbone = u.select_atoms('backbone')

# Align the trajectory by the backbone
aligner = align.AlignTraj(u, u, select='backbone', in_memory=True).run()

# Use the MDAnalysis RMSD analysis class
rmsd_analysis = rms.RMSD(backbone, ref_frame=0)
rmsd_analysis.run()

# Extract the time and RMSD values
frame_nums = range(len(rmsd_analysis.results.rmsd))
rmsd_values = rmsd_analysis.results.rmsd[:, 2]

# Create the plot
plt.figure(figsize=(8, 5))
plt.plot(frame_nums, rmsd_values, 'r-', lw=2)
plt.xlabel('Frame (#)', fontsize=14)
plt.ylabel('RMSD (Å)', fontsize=14)
plt.title('RMSD Over Time', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Get the coordinates of the ligand's first atom.

# Select the first ligand atom
first_atom_lig = u.select_atoms('resname LIG')[0]

# Collect the coordinates
coords = []
for ts in u.trajectory:
    coords.append(first_atom_lig.position)

# Convert it to a numpy array
coords = np.array(coords)

# Get the start coordinates as a reference
start_coors = coords[0]

# Plot the distance between that starting coordinate and the same atom in the
# subsequent frames. Remember the equation for distance:
# https://en.wikipedia.org/wiki/Distance#Straight-line_or_Euclidean_distance

dists = []
for coor in coords:
  # YOUR CALCULATIONS GO HERE
  dist = # YOUR FINAL DISTANCE VALUE GOES HERE
  dists.append(dist)

# Create the plot
plt.figure(figsize=(8, 5))
plt.plot(frame_nums, dists, 'r-', lw=2)
plt.xlabel('Time (ps)', fontsize=14)
plt.ylabel('Distance (Å)', fontsize=14)
plt.title('Distance Over Time', fontsize=16)
plt.grid(True)
plt.show()
